*Is there anything more useless or less usefull than algebra?* \
*- Billy Connoly*

Linear Algebra is the branch of mathematics that deals with *vector spaces* 

# Vectors

Abstractly, *vectors* are objects that can be added together to form new vectors and that can be multiplied by *scalars* (i.e, numbers, also to form new vectors).

Concretely (for us), vectors are points in some finite-dimensional space. Although we might not think of our data as vectors, they are often a useful way to represent numeric data.

For example, if you have the heights, weights, and ages of a large number of people, you can treat your data as three-dimensional vectors [height, weight, age].
If you are teaching a class with four exams, you can treat student grades as four-dimensional vectors [exam1, exam2, exam3, exam4]

The simplest form-scratch approach is to represent vectors as lists of numbers. A list of three numbers corresponds to a vector in three-dimensional space, and vice versa.


We will accomplish this with a type alias that says a Vector is just a list of floats:

In [1]:
from typing import List

vector = List[float]

height_weight_age = [70, # inches
                     170, # pounds
                     40] # years

grades = [95, # exam1
          80, # exam2
          75, # exam3
          62,] # exam4


We will also want to perform *arithmetic* on vectors. Because Python `list` aren't vectors (and hence provide no facilities for vector arithmetic), 
we will need to build these arithmetic tools ourselves.

To begin with, we will frequently need to add two vectors. Vectors add *componentwise*.
This means that if two vectors v and w are the same length, their sum is just the vector whose first element is v[0] + w[0], who's second element is v[1] + w[1] and so on.
(If they are not the same length then we can not add them).

For example, adding the vectors [1, 2] and [2, 1] results in [1 + 2,  2 + 1] or [3, 3].

We can easily do this by zipping them together and using list comprehension to add the corresponding elements:
    

In [3]:
def add(v: vector, w: vector) -> vector:
    """ Adds corresponding elements"""
    assert len(v) == len(w), "Vectors must be the same length"
    return [v_i + w_i for v_i, w_i in zip(v, w)]
assert add ([1, 2, 3], [4, 5, 6]) == [5, 7, 9]

In [4]:
add([1, 2, 3], [4, 5, 6])

[5, 7, 9]

In [5]:
def subtract(v: vector, w: vector) -> vector:
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    return [v_i - w_i for v_i, w_i in zip(v, w)]

assert subtract([5, 7, 9], [4, 5, 6]) == [1, 2, 3]

In [6]:
subtract([5, 7, 9], [4, 5, 6])

[1, 2, 3]

We will sometimes want to componentwise sum a list of vectors - that is - create a new vector whose first element is the sum of all the first elements, whose second element is the sum of all the second elements, and so on:
    

In [7]:
def vector_sum(vectors: list[vector]) -> vector:
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided"
    
    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes"
    
    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
           for i in range(num_elements)]

assert vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]]) == [16, 20]

In [8]:
vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]])

[16, 20]